In [84]:
import pandas as pd

df = pd.read_csv('/Users/ijunhui/Downloads/ubion/프로젝트2/1_기업선정/관리종목/닉변관리종목.csv', encoding='euc-kr')

df = df.dropna()
df.head(10)

,번호,시간,회사명,기간
0,1.0,2012.04.13,JH코오스 | 디테크놀로지,2011.10.16
1,2.0,2012.06.20,엔터미디어 | 엔터기술,2011.12.23
2,3.0,2012.08.14,"""GT&T""",2012.02.16
3,4.0,2012.08.14,티에스넥스젠 | 위드윈네트웍 | 유아이엠엔터 | 에이치엘비파워,2012.02.16
4,5.0,2012.09.20,"""메디콕스""",2012.03.24
5,6.0,2012.09.20,와이오엠 | 스템싸이언스 | 케이엠알앤씨 | 신후,2012.03.24
6,7.0,2012.12.13,네이처셀 | 알앤엘삼미,2012.06.16
7,8.0,2013.03.08,유에이블 | 디웍스글로벌 | 크라제비엠티 | 크라제산업개발,2012.09.09
8,9.0,2013.03.19,아이에이 | 씨앤에스테크놀로지,2012.09.20
9,10.0,2013.03.21,렉스엘이앤지 | 기륭이앤이,2012.09.22


In [114]:
test = "한림대 ‘자랑스러운 Hallymer상’에 장준기 네이버 엔터기술 총괄"
#기업명이 변경된 기업명 중 검색 오류 발생
# '|' 를 넣어 둘 중 하나가 포함된 뉴스 기사가 검색은 되지만 " | " 그 자체가 입력되어 검색되기 때문에 둘 중 하나가 포함된 기사만 선택
# -> 한정된 기간 안애 두 가지 상호의 기업명이 검색됨 -> 데이터 부족으로 이어질 수도
# -> 별개의 기업으로 판단하고 나눠서 검색

for i in range(0, 9):
    if '|' in df.loc[i]['회사명']:
        cm = df.loc[i]['회사명']
        cms = cm.split('|')
        for c in range(0, len(cms)):
            if cms[c] in test:
                print(df.loc[i]['회사명'])
    else:
        print(df.loc[i]['회사명'])


엔터미디어 | 엔터기술
"GT&T"
"메디콕스"


In [29]:
# 구글 크롬 사용을 위해 webdriver 따로 다운로드 후 불러오기
from cgitb import html
import click
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.common.exceptions import NoSuchElementException

# chromedirver 불러오기
path = "/Users/ijunhui/Downloads/ubion/프로젝트2/chromedriver 2"
driver = webdriver.Chrome(path)
driver.maximize_window()
sleep(1)

Url = "https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query"
driver.get(Url)

# 빈 리스트에 저장
com_l = []
news_l = []
title_l = []
time_l = []
text_l = []

d = 79
for i in range(d, 97): # 종목 갯수 처음부터 끝까지
   for j in range(1, 4001, 10):#페이지 1부터 10씩 더해가면서 최대 40페이지까지
      for k in range(j, j+10): #요소 1번부터 끝까지 계속
         Url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+ df.loc[i]['회사명'] +"&sort=2&photo=0&field=0&pd=3&ds="+ df.loc[i]['기간'] +"&de="+ df.loc[i]['시간'] +"&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20110907to20120307,a:all&start=" + str(j)
         driver.get(Url)

         try: # 없는 element 예외처리
            
            
            if '|' in df.loc[i]['회사명']:
               cm = df.loc[i]['회사명']
               cms = cm.split('|')
               for c in range(0, len(cms)):
                  if cms[c] in test:
                     print(df.loc[i]['회사명'])
            else:
               print(df.loc[i]['회사명'])

               
            test = driver.find_element(By.XPATH, """//*[@id='sp_nws""" + str(k) + """']/div/div/a""").text
            if df.loc[i]['회사명'] in test:
               
                  # 네이버 뉴스로 들어가서 제목, 시간, 본문 크롤링
                  news = driver.find_element(By.XPATH, """//*[@id='sp_nws""" + str(k) + """']/div/div/div[1]/div[2]/a[1]""").text

                  tona = driver.find_element(By.XPATH, """//*[@id='sp_nws""" + str(k) + """']/div[1]/div/div[1]/div[2]/a[2]""").click()
                  
                  driver.switch_to.window(driver.window_handles[1])

                  title = driver.find_element(By.XPATH, 
                                                "//*[@id='ct']/div[1]/div[2]").text

                  time = driver.find_element(By.XPATH, 
                                                "//*[@id='ct']/div[1]/div[3]/div[1]/div[1]/span").text
                  text = driver.find_element(By.XPATH, 
                                                "//*[@id='dic_area']").text

                  # sleep(1)

                  driver.close()

                  sleep(1)
                  
                  driver.switch_to.window(driver.window_handles[0])
                  
                  # sleep(1)

                  com_l.append(df.loc[i]['회사명'])
                  news_l.append(news)
                  title_l.append(title)
                  time_l.append(time)
                  text_l.append(text)

                  a = pd.DataFrame({'종목명' : com_l, '신문사' : news_l, '시간' : time_l, '제목' : title_l, '본문' : text_l})
                  a.to_csv('./result/' + str(i) + df.loc[i]['회사명'] +'.csv', index=False)
         
         except NoSuchElementException: 
            pass




NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=105.0.5195.125)


In [8]:
i = 18

h = pd.read_csv('./result/' + str(i)+ df.loc[i]['회사명'] + '.csv')
h

FileNotFoundError: [Errno 2] No such file or directory: './result/18한국테크놀로지.csv'